# Extraction of user song data
- The goal is to be able to select only users who have listened to more than n songs. 
- Then take 10 random out of the n from each user and put it in a file of "known" "inserted" songs. 
- The file of "unknown" "secret" songs is the third output of this notebook 


In [2]:
import os
import pandas as pd
import random
import dplython
import numpy as np
import sys
from sklearn import preprocessing
from sklearn.preprocessing import quantile_transform


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python

In [3]:
#data_dir = '/Users/filiprolenec/Documents/skola/01DAS/01DAS/song-recomendation/Data/MillionSongSubset'
data_dir = os.getcwd()
data_dir

'/Users/filiprolenec/Documents/skola/DAS/Song-Recommender'

In [4]:
user_song_path = os.path.join(data_dir,'data/train_triplets.txt')
user_song_data_original = pd.read_csv(user_song_path, sep="\t", header=None)
user_song_data_original.columns = ["user", "song id", "count"]

In [5]:
number_of_songs_listened = user_song_data_original[['user','song id']].groupby('user').count()
print(number_of_songs_listened['song id'].value_counts().sort_index())



10      50608
11      46219
12      42170
13      38721
14      35585
        ...  
1566        1
1604        1
1614        1
1651        1
4400        1
Name: song id, Length: 920, dtype: int64


In [6]:
users_more_than_n = number_of_songs_listened[number_of_songs_listened['song id']>49]['song id'].keys()

In [7]:
users_50_plus = user_song_data_original[user_song_data_original['user'].isin(users_more_than_n)].sort_values(['user'])

## prokázalo se že se vyplatí zakódovat a zmenšit

In [8]:
le = preprocessing.LabelEncoder()


In [9]:
tmp = le.fit_transform(users_50_plus["user"])

In [10]:
np.save("data/label_encoder_users_classes_.npy",le.classes_)

In [11]:
users_50_plus["user"]=tmp

In [12]:
sys.getsizeof(users_50_plus["user"])*1e-6

508.94129599999997

In [14]:
les = preprocessing.LabelEncoder()

In [15]:
tmps = les.fit_transform(users_50_plus["song id"])

In [16]:
np.save("data/label_encoder_songs_classes_.npy",les.classes_)

In [17]:
sys.getsizeof(users_50_plus["song id"])*1e-6

2640.132839

In [18]:
sys.getsizeof(tmps)*1e-6

254.47072799999998

In [19]:
users_50_plus["song id"]=tmps

In [20]:
sys.getsizeof(users_50_plus)*1e-6

1017.8825599999999

In [21]:
sys.getsizeof(users_50_plus.values)*1e-6

0.000112

In [22]:
np.save("data/users_50_plus_[user, song id, count].npy",users_50_plus.values)

In [23]:
users_50_plus.values

array([[     0,  41258,      1],
       [     0, 220529,      1],
       [     0, 215752,      1],
       ...,
       [289106, 221278,      2],
       [289106, 168955,      1],
       [289106, 204481,      1]])

In [24]:
temp = np.load("data/users_50_plus_[user, song id, count].npy")

In [25]:
temp

array([[     0,  41258,      1],
       [     0, 220529,      1],
       [     0, 215752,      1],
       ...,
       [289106, 221278,      2],
       [289106, 168955,      1],
       [289106, 204481,      1]])

# Now take 10 random out. 

In [26]:
#cca 5 min
size = 10        # sample size
replace = False  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
users_50_plus_train = users_50_plus.groupby('user', as_index=False).apply(fn)

In [27]:
users_50_plus_train

user  song id  count
0      45417979       0   260702      1
       45417954       0   130728      2
       45417966       0   214046      1
       45417984       0   309263      1
       45417959       0   172530      1
...                 ...      ...    ...
289106 8217122   289106   350697      1
       8217075   289106   148683      2
       8217096   289106   228869      2
       8217116   289106   331386      1
       8217079   289106   165925      1

[2891070 rows x 3 columns]

In [28]:
users_50_plus_test = pd.concat([users_50_plus,users_50_plus_train]).drop_duplicates(keep=False)



# Saving the data

In [28]:
file_name_original = 'data/user_50_plus_original.csv' #'/Users/filiprolenec/Documents/skola/01DAS/01DAS/song-recomendation/Data/BaselineModel/
users_50_plus.to_csv(file_name_original, sep=';', index=False)

SystemError: <built-in method item of numpy.ndarray object at 0x1844183a0> returned a result with an error set

In [ ]:
file_name_test = 'data/user_50_plus_test.csv' #/Users/filiprolenec/Documents/skola/01DAS/01DAS/song-recomendation/Data/BaselineModel/
users_50_plus_test.to_csv(file_name_test, sep=';', index=False)

In [ ]:
file_name_train = 'data/user_50_plus_train.csv' # /Users/filiprolenec/Documents/skola/01DAS/01DAS/song-recomendation/Data/BaselineModel/
users_50_plus_train.to_csv(file_name_train, sep=';', index=False)

# Getting the codes for top 10 songs. 


In [29]:
top_10_listened = pd.read_csv("data/top_10_listened_songs.csv", sep = ";")

In [30]:
top_10_listened['song id']

0    SOULTKQ12AB018A183
1    SOIZLKI12A6D4F7B61
2    SOEHWGF12A6D4F8B2B
3    SOEKSGJ12A67AE227E
4    SOUCKDH12A8C138FF5
5    SONQBUB12A6D4F8ED0
6    SOHTQAS12A6701C7BA
7    SOBEVGM12A67ADBCA7
8    SOKXYUW12A8C140229
9    SOSQIHH12A8C13370B
Name: song id, dtype: object

In [31]:
le = preprocessing.LabelEncoder()

In [32]:
le.classes_ = np.load("data/label_encoder_songs_classes_.npy", allow_pickle = True)


In [33]:
top_10_encoded=le.transform(top_10_listened['song id'])

# mAP declaration


In [34]:
def APn(orig,predicted, n):
    """
    orig: list, order doesnt matter
    predicted: list, order matters
    n: integer, truncation parameter
    
    Output
    ap: double, the average precision over the input item lists
    """
    if len(predicted) > n:
        predicted = predicted[:n]
    ap = 0.0
    m = 0.0 #number of hits
    
    for k,l in enumerate(predicted):
        if l in orig and l not in predicted[:k]:
            m += 1.0
            ap += m / (k+1.0)
        
    if not orig:
        return 0.0
    
    return ap / min(len(orig),n)

In [35]:
def mAPn(orig, predicted, n):
    """
    truncated Mean Average Precision
    """
    
    return np.mean([APn(o,pred,n) for o, pred in zip(orig,predicted)])

# Baseline model - mAP computed... 

In [36]:
users_50_plus_train[0:20]

user  song id  count
0 45417979     0   260702      1
  45417954     0   130728      2
  45417966     0   214046      1
  45417984     0   309263      1
  45417959     0   172530      1
  45417947     0   105638      1
  45417946     0   102787      2
  45417935     0    26014      2
  45417970     0   227339      1
  45417960     0   184133      1
1 39783461     1   163885      2
  39783423     1    59487      2
  39783411     1    32438      1
  39783477     1   193436      1
  39783532     1   325123      2
  39783463     1   166498      1
  39783458     1   153385      1
  39783465     1   172147      1
  39783540     1   345801      1
  39783441     1   116570      3

In [37]:
users_50_plus_test = users_50_plus_test.sort_values(by = ['user','count'], ascending=[True,False])

In [38]:
def baseline_recommend(known_10):
    return top_10_encoded.tolist()

In [39]:
# 290k values
grouped =users_50_plus_test.groupby('user')
apn_users = {}
for name, group in grouped:
    first_20_unknown = group.iloc[1:20,]['song id'].tolist()
    known_10 = users_50_plus_train[name*10:(name+1)*10-1]['song id'].tolist()
    recommended_10 = baseline_recommend(known_10)
    
    apn_users[name] = APn(first_20_unknown,recommended_10,10)
    
    if(name%100==0):
        print(name)
    
final_apn_average = np.array(list(apn_users.values())).mean()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

133000
133100
133200
133300
133400
133500
133600
133700
133800
133900
134000
134100
134200
134300
134400
134500
134600
134700
134800
134900
135000
135100
135200
135300
135400
135500
135600
135700
135800
135900
136000
136100
136200
136300
136400
136500
136600
136700
136800
136900
137000
137100
137200
137300
137400
137500
137600
137700
137800
137900
138000
138100
138200
138300
138400
138500
138600
138700
138800
138900
139000
139100
139200
139300
139400
139500
139600
139700
139800
139900
140000
140100
140200
140300
140400
140500
140600
140700
140800
140900
141000
141100
141200
141300
141400
141500
141600
141700
141800
141900
142000
142100
142200
142300
142400
142500
142600
142700
142800
142900
143000
143100
143200
143300
143400
143500
143600
143700
143800
143900
144000
144100
144200
144300
144400
144500
144600
144700
144800
144900
145000
145100
145200
145300
145400
145500
145600
145700
145800
145900
146000
146100
146200
146300
146400
146500
146600
146700
146800
146900
147000
147100
147200

250000
250100
250200
250300
250400
250500
250600
250700
250800
250900
251000
251100
251200
251300
251400
251500
251600
251700
251800
251900
252000
252100
252200
252300
252400
252500
252600
252700
252800
252900
253000
253100
253200
253300
253400
253500
253600
253700
253800
253900
254000
254100
254200
254300
254400
254500
254600
254700
254800
254900
255000
255100
255200
255300
255400
255500
255600
255700
255800
255900
256000
256100
256200
256300
256400
256500
256600
256700
256800
256900
257000
257100
257200
257300
257400
257500
257600
257700
257800
257900
258000
258100
258200
258300
258400
258500
258600
258700
258800
258900
259000
259100
259200
259300
259400
259500
259600
259700
259800
259900
260000
260100
260200
260300
260400
260500
260600
260700
260800
260900
261000
261100
261200
261300
261400
261500
261600
261700
261800
261900
262000
262100
262200
262300
262400
262500
262600
262700
262800
262900
263000
263100
263200
263300
263400
263500
263600
263700
263800
263900
264000
264100
264200

In [40]:
final_apn_average

0.0022482609421095863

In [42]:
users_50_plus_train

user  song id  count
0      45417931       0     1060      1
       45417963       0   193080      1
       45417935       0    26014      2
       45417958       0   163733      4
       45417985       0   312677      1
...                 ...      ...    ...
289106 8217053   289106    30220      4
       8217095   289106   228585      1
       8217064   289106    76345      2
       8217085   289106   199895      2
       8217117   289106   332619      2

[2891070 rows x 3 columns]

In [43]:
users_50_plus_train.reset_index(drop=True, inplace=True)
us

NameError: name 'us' is not defined

# Transformation count -> stars

Toto je docela pomale - pokud nekoho napadne jak zrychlit budu rad... 

In [61]:
#290k values
grouped = users_50_plus_train.groupby('user')
users_50_plus_train['stars']=None
stars =[]
for name, group in grouped:  
    stars_current=pd.cut(list((group['count'])), bins=5, labels=[1,2,3,4,5])
    #print(list(stars_current))
    stars = np.concatenate([stars,list(stars_current)])
    #users_50_plus_train['stars'][indexes]=group['stars'][indexes]
    if(name%100==0):
        print(name)
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

133000
133100
133200
133300
133400
133500
133600
133700
133800
133900
134000
134100
134200
134300
134400
134500
134600
134700
134800
134900
135000
135100
135200
135300
135400
135500
135600
135700
135800
135900
136000
136100
136200
136300
136400
136500
136600
136700
136800
136900
137000
137100
137200
137300
137400
137500
137600
137700
137800
137900
138000
138100
138200
138300
138400
138500
138600
138700
138800
138900
139000
139100
139200
139300
139400
139500
139600
139700
139800
139900
140000
140100
140200
140300
140400
140500
140600
140700
140800
140900
141000
141100
141200
141300
141400
141500
141600
141700
141800
141900
142000
142100
142200
142300
142400
142500
142600
142700
142800
142900
143000
143100
143200
143300
143400
143500
143600
143700
143800
143900
144000
144100
144200
144300
144400
144500
144600
144700
144800
144900
145000
145100
145200
145300
145400
145500
145600
145700
145800
145900
146000
146100
146200
146300
146400
146500
146600
146700
146800
146900
147000
147100
147200

250100
250200
250300
250400
250500
250600
250700
250800
250900
251000
251100
251200
251300
251400
251500
251600
251700
251800
251900
252000
252100
252200
252300
252400
252500
252600
252700
252800
252900
253000
253100
253200
253300
253400
253500
253600
253700
253800
253900
254000
254100
254200
254300
254400
254500
254600
254700
254800
254900
255000
255100
255200
255300
255400
255500
255600
255700
255800
255900
256000
256100
256200
256300
256400
256500
256600
256700
256800
256900
257000
257100
257200
257300
257400
257500
257600
257700
257800
257900
258000
258100
258200
258300
258400
258500
258600
258700
258800
258900
259000
259100
259200
259300
259400
259500
259600
259700
259800
259900
260000
260100
260200
260300
260400
260500
260600
260700
260800
260900
261000
261100
261200
261300
261400
261500
261600
261700
261800
261900
262000
262100
262200
262300
262400
262500
262600
262700
262800
262900
263000
263100
263200
263300
263400
263500
263600
263700
263800
263900
264000
264100
264200
264300

In [62]:
users_50_plus_train['stars']=stars

In [63]:
print(users_50_plus_train.iloc[999:1200].to_string())

      user  song id  count  stars
999     99    77760      1    1.0
1000   100   351365      1    1.0
1001   100    82276      1    1.0
1002   100   281087      1    1.0
1003   100   261205      1    1.0
1004   100    79807      2    2.0
1005   100   315022      1    1.0
1006   100   215830      1    1.0
1007   100   145006      5    5.0
1008   100   230326      1    1.0
1009   100    38323      2    2.0
1010   101   203264      3    2.0
1011   101    86340      1    1.0
1012   101   328588      3    2.0
1013   101    87318      3    2.0
1014   101   177201      6    5.0
1015   101   173500      4    3.0
1016   101     3694      1    1.0
1017   101   266559      4    3.0
1018   101   188465      5    4.0
1019   101    50166      4    3.0
1020   102   312652      1    1.0
1021   102   257660      1    1.0
1022   102   225927     13    5.0
1023   102   218655      1    1.0
1024   102   113087      1    1.0
1025   102    34013      1    1.0
1026   102   333560      1    1.0
1027   102   1

# End of the code
## Now it can
- Change count -> stars, 
- Compute map in a way we want to (Kajetan will make an update)
- Compute map for baseline model - 0.0024957619146506534
## What is missing
- Real model, that can recommend something non-trivial...
- Its MAP